In [5]:
pip install cryptography

In [ ]:
import math
import random
from typing import Tuple

def generate_prime(min_value: int, max_value: int) -> int:
    """Generate a prime number between min_value and max_value."""
    def is_prime(n: int) -> bool:
        if n < 2:
            return False
        for i in range(2, int(math.sqrt(n)) + 1):
            if n % i == 0:
                return False
        return True

    prime = random.randrange(min_value, max_value)
    while not is_prime(prime):
        prime = random.randrange(min_value, max_value)
    return prime

def generate_keypair(p: int, q: int) -> Tuple[Tuple[int, int], Tuple[int, int]]:
    """Generate public and private keypairs."""
    n = p * q
    phi = (p - 1) * (q - 1)

    # Choose e: coprime to phi and 1 < e < phi
    e = random.randrange(1, phi)
    while math.gcd(e, phi) != 1:
        e = random.randrange(1, phi)

    # Calculate d: modular multiplicative inverse of e
    def extended_gcd(a: int, b: int) -> Tuple[int, int, int]:
        if a == 0:
            return b, 0, 1
        gcd, x1, y1 = extended_gcd(b % a, a)
        x = y1 - (b // a) * x1
        y = x1
        return gcd, x, y

    _, d, _ = extended_gcd(e, phi)
    d = d % phi
    if d < 0:
        d += phi

    return ((e, n), (d, n))

def encrypt(public_key: Tuple[int, int], plaintext: str) -> list:
    """Encrypt the plaintext using public key."""
    e, n = public_key
    # Convert each character to number and encrypt
    cipher = [(ord(char) ** e) % n for char in plaintext]
    return cipher

def decrypt(private_key: Tuple[int, int], ciphertext: list) -> str:
    """Decrypt the ciphertext using private key."""
    d, n = private_key
    # Decrypt each number and convert back to character
    plain = [chr((char ** d) % n) for char in ciphertext]
    return ''.join(plain)

def main():
    # Generate two prime numbers
    p = generate_prime(100, 1000)
    q = generate_prime(100, 1000)

    print(f"Generated prime numbers: p = {p}, q = {q}")

    # Generate public and private keys
    public_key, private_key = generate_keypair(p, q)
    print(f"Public key: {public_key}")
    print(f"Private key: {private_key}")

    # Get message from user
    message = input("Enter a message to encrypt: ")

    # Encrypt the message
    encrypted_msg = encrypt(public_key, message)
    print(f"Encrypted message: {encrypted_msg}")

    # Decrypt the message
    decrypted_msg = decrypt(private_key, encrypted_msg)
    print(f"Decrypted message: {decrypted_msg}")

if __name__ == "__main__":
    main()

Generated prime numbers: p = 821, q = 241
Public key: (182143, 197861)
Private key: (180607, 197861)


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import random
from sympy import mod_inverse
from PIL import Image
import numpy as np

def generate_keys(p, q):
    n = p * q
    φ_n = (p - 1) * (q - 1)

    e = random.randint(2, φ_n - 1)
    while gcd(e, φ_n) != 1:
        e = random.randint(2, φ_n - 1)

    d = mod_inverse(e, φ_n)
    return (e, n), (d, n)

def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

def encrypt_image(image_bytes, public_key):
    e, n = public_key
    encrypted_data = []

    # Encrypt each byte chunk using RSA
    for byte in image_bytes:
        encrypted_byte = pow(byte, e, n)
        encrypted_data.append(encrypted_byte)

    return encrypted_data

def decrypt_image(encrypted_data, private_key):
    d, n = private_key
    decrypted_data = []

    # Decrypt each byte chunk using RSA
    for encrypted_byte in encrypted_data:
        decrypted_byte = pow(encrypted_byte, d, n)
        decrypted_data.append(decrypted_byte)

    return decrypted_data

def save_encrypted_image(encrypted_data, image_shape, output_filename):
    encrypted_array = np.array(encrypted_data, dtype=np.uint8).reshape(image_shape)
    encrypted_image = Image.fromarray(encrypted_array)
    encrypted_image.save(output_filename)

def load_image_as_bytes(image_path):
    image = Image.open(image_path).convert('L')  # Convert image to grayscale for simplicity
    image_data = np.array(image)
    image_bytes = image_data.flatten().tolist()
    return image_bytes, image_data.shape

def save_decrypted_image(decrypted_data, image_shape, output_filename):
    decrypted_array = np.array(decrypted_data, dtype=np.uint8).reshape(image_shape)
    decrypted_image = Image.fromarray(decrypted_array)
    decrypted_image.save(output_filename)

# Main program execution
p = 101  # Choose large primes for real-world security
q = 103
public_key, private_key = generate_keys(p, q)

print(f"Public Key: {public_key}")
print(f"Private Key: {private_key}")

# Load the image and get bytes
image_path = "/content/images.jpeg"  # Replace with your image path
image_bytes, image_shape = load_image_as_bytes(image_path)

# Encrypt the image
encrypted_data = encrypt_image(image_bytes, public_key)

# Save encrypted image for visualization (optional)
save_encrypted_image(encrypted_data, image_shape, "/content/encrypted_image.png")
print("Encrypted image saved as 'encrypted_image.png'")

# Decrypt the image
decrypted_data = decrypt_image(encrypted_data, private_key)

# Save decrypted image
save_decrypted_image(decrypted_data, image_shape, "/content/decrypted_image.png")
print("Decrypted image saved as 'decrypted_image.png'")
